In [27]:
import pandas as pd
import numpy as np
import glob
import warnings
warnings.filterwarnings('ignore')

In [28]:
raw_dir = '../data/raw'
interim_dir = '../data/interim/'
external_dir = '../data/external'

In [29]:
source_files = pd.DataFrame({'file_path' : sorted(glob.glob(f'{raw_dir}/*.xlsx'))})

In [30]:
source_files['variable_type'] = \
    source_files['file_path'] \
        .replace('../data/raw/[0-9][a-z] - ', '', regex=True) \
            .replace('.xlsx', '', regex=True).replace('\W{1,}', '_', regex=True) \
                .str.lower()

In [31]:
dfs = {}
for i in source_files.index:
    dfs[source_files.loc[i,'variable_type']] =  pd.read_excel(source_files.loc[i,'file_path'])

In [32]:
population = pd.read_csv(f'{external_dir}/WPP2022_Demographic_Indicators_Medium.csv', usecols=['Location', 'Time', 'TPopulation1Jan', 'TPopulation1July'])
population = population.rename(columns = {'Location' : 'country', 'Time' : 'year', 'TPopulation1Jan' : 'population_boy', 'TPopulation1July' : 'population_mid'})


In [33]:
def process_producer(df, var_name, time_name):
    """ Processes producer data
        df: pandas DataFrame containing raw producer data
        var_name:   variable name to process (production, consumption, openstock, or exports)
        time:name:  crop_year or calendar_year
    """
    x = df
    x.columns = x.iloc[2, :]
    x = x.iloc[3:(x.shape[0]-1), :]
    x = x.dropna(how='all')
    x = x.loc[:,x.columns.notna()]
    x.rename(columns={'Crop year' : 'country', 'Crop years' : 'country', 'Calendar years' : 'country'}, inplace=True)
    x = x.melt(id_vars='country', var_name = time_name, value_name=f'{var_name}_1k_bags')
    x['country'] = x['country'].str.strip()
    x[f'{var_name}_1k_bags'] = x[f'{var_name}_1k_bags'].astype('float', errors='ignore')
    x[f'{var_name}_kg'] = x[f'{var_name}_1k_bags'] * 1000 * 60
    x[f'{var_name}_lb'] = x[f'{var_name}_kg'] * 2.20462262185
    harvest_groups = ['April group', 'July group', 'October group']
    totals = ['Total']
    x['harvest_group'] = np.where(x['country'].isin(harvest_groups), x['country'].str.replace(r' [Gg]roup', '', regex=True), np.NaN)
    x['harvest_group'] = x['harvest_group'].ffill()
    if time_name == 'crop_year':
        x['crop_year_beg'] = x['crop_year'].str[0:4].astype('int')
        x['crop_year_end'] = x['crop_year_beg']+1
    hg_rows = x.iloc[:,0].isin(harvest_groups)
    tot_rows = x.iloc[:,0].isin(totals)
    x = x.loc[~(hg_rows|tot_rows)]
    if time_name == 'crop_year':
        x = x[['country', 'harvest_group','crop_year', 'crop_year_beg', 'crop_year_end',f'{var_name}_1k_bags',f'{var_name}_kg',f'{var_name}_lb']]
    else:
        x = x[['country', 'calendar_year',f'{var_name}_1k_bags',f'{var_name}_kg',f'{var_name}_lb']]
    return(x)

In [34]:
dfs['total_production'] = process_producer(dfs['total_production'], 'production', 'crop_year')
dfs['domestic_consumption'] = process_producer(dfs['domestic_consumption'], 'consumption', 'crop_year')
dfs['gross_opening_stocks'] = process_producer(dfs['gross_opening_stocks'], 'openstock', 'crop_year')
dfs['exports_crop_year'] = process_producer(dfs['exports_crop_year'], 'exports', 'crop_year')
dfs['exports_calendar_year'] = process_producer(dfs['exports_calendar_year'], 'exports', 'calendar_year')


In [35]:
region_map = pd.DataFrame(
    {'country' : ['Austria','Belgium','Belgium/Luxembourg','Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','Finland','France','Germany',
'Greece','Hungary','Ireland','Italy','Latvia','Lithuania','Luxembourg','Malta','Netherlands','Poland','Portugal',
'Romania','Slovakia','Slovenia','Spain','Sweden','Japan','Norway','Russian Federation','Switzerland','Tunisia','United Kingdom','United States of America']
,
'region' : ['Europe','Europe','Europe','Europe','Europe','Europe','Europe','Europe','Europe','Europe','Europe','Europe','Europe','Europe',
'Europe','Europe','Europe','Europe','Europe','Europe','Europe','Europe','Europe','Europe','Europe','Europe','Europe',
'Europe','Asia & Oceania','Europe','Europe','Europe','Africa','Europe','North America']
    }
)

In [36]:
def process_importer(df, var_name, region_map):
    x = df
    x.columns = x.iloc[2, :]
    x = x.iloc[3:43, :]
    x = x.dropna(how='all')
    x.rename(columns={'Calendar years' : 'country'}, inplace=True)
    x = x.melt(id_vars='country', var_name = 'calendar_year', value_name=f'{var_name}_1k_bags')
    x['country'] = x['country'].str.strip()
    x[f'{var_name}_1k_bags'] = x[f'{var_name}_1k_bags'].astype('float', errors='ignore')
    eu = ['European Union']
    eu_rows = x.iloc[:,0].isin(eu)
    totals = ['Total']
    tot_rows = x.iloc[:,0].isin(totals)
    x = x.loc[~(eu_rows|tot_rows)]
    x = x.merge(region_map, how = 'left', on = 'country')
    x['ico_member'] = 'member'

    B = x.loc[lambda x: x.country.isin(['Belgium']) & (x.calendar_year >= 1999), ['calendar_year', f'{var_name}_1k_bags']]
    L = x.loc[lambda x: x.country.isin(['Luxembourg']) & (x.calendar_year >= 1999), ['calendar_year', f'{var_name}_1k_bags']]

    BL = \
        B \
            .merge(L, how = 'left', on = 'calendar_year', suffixes=['_B', '_L']) \
                .assign(tot = lambda x: x[f'{var_name}_1k_bags_B'] + x[f'{var_name}_1k_bags_L'],
                        B_ratio = lambda x: x[f'{var_name}_1k_bags_B']/x.tot)

    belg_ratio = BL[f'{var_name}_1k_bags_B'].sum() / BL.tot.sum()

    calced_split = \
    x \
    .loc[lambda x: x.country.isin(['Belgium/Luxembourg']), ['calendar_year', f'{var_name}_1k_bags']] \
    .assign(**{f'{var_name}_1k_bags_B' : lambda x: x[f'{var_name}_1k_bags'] * belg_ratio,
                f'{var_name}_1k_bags_L' : lambda x: x[f'{var_name}_1k_bags'] * (1-belg_ratio)})

    B = calced_split[['calendar_year', f'{var_name}_1k_bags_B']] \
        .assign(country = 'Belgium') \
            .rename(columns={f'{var_name}_1k_bags_B' : f'{var_name}_1k_bags'})
    
    L = calced_split[['calendar_year', f'{var_name}_1k_bags_L']] \
        .assign(country = 'Luxembourg') \
            .rename(columns={f'{var_name}_1k_bags_L' : f'{var_name}_1k_bags'})
    
    BL = pd.concat([B, L]).reset_index(drop=True)[['calendar_year', 'country', f'{var_name}_1k_bags']]

    x = \
    x \
    .merge(BL, how = 'left', on = ['calendar_year', 'country'], suffixes = ['', '_calc']) \
    .assign(**{f'{var_name}_1k_bags' : lambda x: np.where(x[f'{var_name}_1k_bags'].isna(), x[f'{var_name}_1k_bags_calc'], x[f'{var_name}_1k_bags'])}) \
    .loc[lambda x: ~x.country.isin(['Belgium/Luxembourg']), 
        ['region', 'country', 'ico_member', 'calendar_year', f'{var_name}_1k_bags']]

    x[f'{var_name}_kg'] = x[f'{var_name}_1k_bags'] * 1000 * 60
    x[f'{var_name}_lb'] = x[f'{var_name}_kg'] * 2.20462262185

    x = x[['region', 'country','ico_member', 'calendar_year', f'{var_name}_1k_bags', f'{var_name}_kg', f'{var_name}_lb']]
    
    return(x)

In [37]:
dfs['imports'] = process_importer(dfs['imports'], 'imports')
dfs['re_exports'] = process_importer(dfs['re_exports'], 're_exports')

In [38]:
member = \
dfs['imports'] \
    .merge(dfs['re_exports'], how = 'outer', on = ['region', 'country', 'ico_member', 'calendar_year'])

In [39]:
def process_nonmember(df, var_name):
    x = df
    x.columns = x.iloc[2, :]
    x = x.iloc[3:(df.shape[0]-1), :]
    x = x.dropna(how='all')
    x.rename(columns={'Calendar years' : 'country'}, inplace=True)
    x = x.melt(id_vars='country', var_name = 'calendar_year', value_name=f'{var_name}_1k_bags')
    x['country'] = x['country'].str.strip()
    x[f'{var_name}_1k_bags'] = x[f'{var_name}_1k_bags'].astype('float', errors='ignore')
    x[f'{var_name}_kg'] = x[f'{var_name}_1k_bags'] * 1000 * 60
    x[f'{var_name}_lb'] = x[f'{var_name}_kg'] * 2.20462262185
    china_agg = x.iloc[:,0].isin(["China, People's Republic of"])
    x = x[~china_agg]
    regions = ['Africa','Asia & Oceania','Caribbean','Central America & Mexico','Europe','North America','South America']
    totals = ['Total']
    x['region'] = np.where(x['country'].isin(regions), x['country'], np.NaN)
    x['region'] = x['region'].ffill()
    x['ico_member'] = 'non-member'
    region_rows = x.iloc[:,0].isin(regions)
    tot_rows = x.iloc[:,0].isin(totals)
    x = x.loc[~(region_rows|tot_rows)]
    x = x[['region','country','ico_member','calendar_year',f'{var_name}_1k_bags',f'{var_name}_kg',f'{var_name}_lb']]
    return(x)

In [40]:
dfs['non_member_imports'] = process_nonmember(dfs['non_member_imports'], 'imports')
dfs['non_member_re_exports'] = process_nonmember(dfs['non_member_re_exports'], 're_exports')

In [41]:
non_member = \
dfs['non_member_imports'] \
    .merge(dfs['non_member_re_exports'], how = 'outer', on = ['region', 'country', 'ico_member', 'calendar_year'])

In [86]:
imports_re_exports = pd.concat([member, non_member]).reset_index(drop=True)

In [87]:
fmr_yugoslavia = \
    population \
        .loc[lambda x: x.country.isin(['Serbia', 'Croatia', 'Slovenia', 'Bosnia and Herzegovina', 'Macedonia'])] \
        .groupby('year') \
        .agg({'population_boy' : sum, 'population_mid' : sum}) \
        .reset_index() \
        .assign(country = 'Yugoslavia SFR') \
        .loc[:, ['country', 'year', 'population_boy', 'population_mid']]

net_antilles = \
    population \
        .loc[lambda x: x.country.isin(['Curacao', 'Bonaire', 'Aruba', 'Sint Maarten (Dutch part)', 'Sint Eustatius', 'Saba', 'Netherlands Antilles'])] \
        .groupby('year') \
        .agg({'population_boy' : sum, 'population_mid' : sum}) \
        .reset_index() \
        .assign(country = 'Netherlands Antilles (former)') \
        .loc[:, ['country', 'year', 'population_boy', 'population_mid']]

population = pd.concat([population, fmr_yugoslavia, net_antilles])

In [88]:
countrymap = \
pd.DataFrame(
    {
        'country' : 
            ['Abu Dhabi',
            'China (Mainland)',
            "Democratic People's Republic of Korea",
            'Dubai',
            'Hong Kong',
            'Macao',
            'Micronesia (Federated States of)',
            'Netherlands Antilles (former)',
            'Saint Vincent & the Grenadines',
            'Taiwan',
            'Turkey',
            'USSR'],

        'country_pop' :
            ['United Arab Emirates',
            'China',
            "Dem. People's Republic of Korea",
            'United Arab Emirates',
            'China, Hong Kong SAR',
            'China, Macao SAR',
            'Micronesia',
            'Netherlands Antilles (former)',
            'Saint Vincent and the Grenadines',
            'China, Taiwan Province of China',
            'Türkiye',
            'Russian Federation']
    }
)

In [89]:
imports_re_exports = \
imports_re_exports \
    .merge(countrymap, how = 'left', on='country') \
    .assign(country_pop = lambda x: np.where(x.country_pop.isna(), x.country, x.country_pop)) 

imports_re_exports = \
    imports_re_exports \
        .merge(population, how = 'left', left_on=['country_pop', 'calendar_year'], right_on = ['country', 'year'], suffixes = ['', '_y']) \
            .drop(columns=['year', 'country_y'])
            
imports_re_exports.drop(columns='country_pop', inplace=True)

In [48]:
x = dfs['prices_paid_to_growers']
x.columns = x.iloc[2, :]
x = x.iloc[3:76, :]
x = x.dropna(how='all')
x.rename(columns={'Calendar years' : 'country'}, inplace=True)
x = x.melt(id_vars='country', var_name = 'calendar_year', value_name='price_paid_cents_per_lb')
x['country'] = x['country'].str.strip()
x['price_paid_cents_per_lb'] = x['price_paid_cents_per_lb'].astype('float', errors='ignore')
x['price_paid_dollars_per_lb'] = x['price_paid_cents_per_lb'] / 100
categories = ['Colombian Milds', 'Other Milds', 'Brazilian Naturals', 'Robustas']
totals = ['Total', '']
x['indicator_name'] = np.where(x['country'].isin(categories), x['country'], np.NaN)
x['indicator_name'] = x['indicator_name'].ffill()
cat_rows = x.iloc[:,0].isin(categories)
tot_rows = x.iloc[:,0].isin(totals)
x = x.loc[~(cat_rows|tot_rows)]
x['indicator_name'] = x['indicator_name'].str.replace(' ', '_').str.lower()
x = x[['country', 'indicator_name', 'calendar_year', 'price_paid_cents_per_lb', 'price_paid_dollars_per_lb']]
dfs['prices_paid_to_growers'] = x

In [49]:
x = dfs['indicator_prices']
x.columns = x.iloc[2, :]
x = x.iloc[3:424, :]
x = x.dropna(how='all')
x.columns = x.columns.fillna('calendar_month').str.replace('\\n', '', regex=True).str.replace(' ', '_', regex=True).str.lower()
x = x.melt(id_vars='calendar_month', var_name='indicator_name', value_name='indicator_price_cents_per_lb')
x['indicator_price_cents_per_lb'] = x['indicator_price_cents_per_lb'].astype('float', errors = 'ignore')
x['indicator_price_dollars_per_lb'] = x['indicator_price_cents_per_lb'] / 100
year_rows = x['calendar_month'].astype('str').str.match(r'[0-9]{4}')
x['calendar_year'] = np.where(year_rows, x['calendar_month'], np.NaN)
x['calendar_year'] = x['calendar_year'].fillna(method='ffill')
avg_annual = x[year_rows].drop(columns='calendar_month')
x = x[~year_rows]
x = x.merge(avg_annual, how = 'left', on = ['calendar_year', 'indicator_name'], suffixes=['', '_ann'])
x = x[['calendar_year','calendar_month','indicator_name','indicator_price_cents_per_lb','indicator_price_dollars_per_lb', 'indicator_price_cents_per_lb_ann','indicator_price_dollars_per_lb_ann']]
dfs['indicator_prices'] = x

### Print Interim Files

In [ ]:
for k in dfs.keys():
    dfs[k].to_csv(f'{interim_dir}/{k}.csv', index=False)

In [94]:
### Merge Producing Country Crop Year Data
producer_cropyear = \
    dfs['gross_opening_stocks'] \
        .merge(dfs['total_production'], 
        how = 'left', 
        on = ['country', 'harvest_group','crop_year', 'crop_year_beg', 'crop_year_end'])

producer_cropyear = \
    producer_cropyear \
        .merge(dfs['domestic_consumption'], 
        how = 'left', 
        on = ['country', 'harvest_group','crop_year', 'crop_year_beg', 'crop_year_end'])
        
producer_cropyear = \
    producer_cropyear \
        .merge(dfs['exports_crop_year'], 
        how = 'left', 
        on = ['country', 'harvest_group','crop_year', 'crop_year_beg', 'crop_year_end'])

In [95]:
## calculated closing stock equals opening stock plus production, less consumption and exports
def close_stock_calc(df, scale):
    return(
        np.fmax(0, 
            np.round(
                df[f'openstock_{scale}'] 
                + df[f'production_{scale}']
                - df[f'consumption_{scale}']
                - df[f'exports_{scale}']
            , 2)
        )
    )

## closing stock based on time-shifted opening stocks
def close_stock_shift(df, scale):
    open_shift = \
    df \
        .groupby('country')\
            [f'openstock_{scale}'] \
                .shift(-1)
    return(open_shift)

## stock adjustment -- difference between calculated and shifted closing stocks
def stock_adj(df, scale):
    return(close_stock_shift(df,scale) - close_stock_calc(df, scale))

In [96]:
producer_cropyear['closestock_1k_bags'] = close_stock_calc(producer_cropyear, '1k_bags')
producer_cropyear['closestock_kg'] = close_stock_calc(producer_cropyear, 'kg')
producer_cropyear['closestock_lb'] = close_stock_calc(producer_cropyear, 'lb')

In [97]:
producer_cropyear.sort_values(['country', 'crop_year_beg'], inplace=True)
producer_cropyear['stock_adj_1k_bags'] = stock_adj(producer_cropyear, '1k_bags')
producer_cropyear['stock_adj_kg'] = stock_adj(producer_cropyear, 'kg')
producer_cropyear['stock_adj_lb'] = stock_adj(producer_cropyear, 'lb')

In [98]:
countrymap = pd.DataFrame(
    {
    'country' : ['Democratic Republic of Congo', 'Tanzania', 'Trinidad & Tobago', 'Venezuela'],
    'country_pop' : ['Democratic Republic of the Congo', 'United Republic of Tanzania', 'Trinidad and Tobago', 'Venezuela (Bolivarian Republic of)']
    }
)

In [99]:
producer_cropyear = \
    producer_cropyear \
        .merge(countrymap, how = 'left', on='country', suffixes = ['', '_pop']) \
            .assign(country_pop = lambda x: np.where(x.country_pop.isna(), x.country, x.country_pop))

producer_cropyear = \
    producer_cropyear \
        .merge(population, how = 'left', left_on=['country_pop', 'crop_year_beg'], right_on = ['country', 'year'], suffixes = ['', '_y']) \
            .drop(columns=['country_y', 'year']) \
                .rename(columns={'population_boy' : 'population_beg'})

producer_cropyear = \
    producer_cropyear \
        .merge(population, how = 'left', left_on=['country_pop', 'crop_year_end'], right_on = ['country', 'year'], suffixes = ['', '_y']) \
            .drop(columns=['country_y', 'year', 'population_mid_y']) \
                .rename(columns={'population_boy' : 'population_end'})

producer_cropyear.drop(columns='country_pop', inplace=True)


In [100]:
exports_calendar_year = \
dfs['exports_calendar_year'] \
    .merge(countrymap, how = 'left', on='country', suffixes = ['', '_pop']) \
    .assign(country_pop = lambda x: np.where(x.country_pop.isna(), x.country, x.country_pop)) 

exports_calendar_year = \
    exports_calendar_year \
        .merge(population, how = 'left', left_on=['country_pop', 'calendar_year'], right_on = ['country', 'year'], suffixes = ['', '_y']) \
            .drop(columns=['country_y', 'year'])
            
exports_calendar_year.drop(columns='country_pop', inplace=True)

In [117]:
annual_indicators = \
dfs['indicator_prices'] \
    .loc[
        lambda x: x.calendar_month == 'January', 
        [
            'calendar_year',
            'indicator_name',
            'indicator_price_cents_per_lb_ann',
            'indicator_price_dollars_per_lb_ann'
        ]
    ]

annual_indicators['calendar_year'] = annual_indicators['calendar_year'].astype('int64')

grower_vs_indicator = dfs['prices_paid_to_growers']
grower_vs_indicator['calendar_year'] = grower_vs_indicator['calendar_year'].astype('int64')

grower_vs_indicator = \
    grower_vs_indicator \
    .merge(
        annual_indicators,
        how='left',
        on=['indicator_name', 'calendar_year']
    )

In [129]:
grower_vs_indicator.loc[lambda x: (x.country == 'Viet Nam') & (x.indicator_name == 'brazilian_naturals')].sort_values(['country', 'calendar_year'])

,country,indicator_name,calendar_year,price_paid_cents_per_lb,price_paid_dollars_per_lb,indicator_price_cents_per_lb_ann,indicator_price_dollars_per_lb_ann
36,Viet Nam,brazilian_naturals,1990,NaN,NaN,82.97,0.8297
101,Viet Nam,brazilian_naturals,1991,NaN,NaN,72.91,0.7291
166,Viet Nam,brazilian_naturals,1992,NaN,NaN,56.49,0.5649
231,Viet Nam,brazilian_naturals,1993,NaN,NaN,66.58,0.6658
296,Viet Nam,brazilian_naturals,1994,NaN,NaN,143.24,1.4324
361,Viet Nam,brazilian_naturals,1995,NaN,NaN,145.95,1.4595
426,Viet Nam,brazilian_naturals,1996,NaN,NaN,119.77,1.1977
491,Viet Nam,brazilian_naturals,1997,NaN,NaN,166.80,1.6680
556,Viet Nam,brazilian_naturals,1998,NaN,NaN,121.81,1.2181
621,Viet Nam,brazilian_naturals,1999,NaN,NaN,88.84,0.8884
